In [ ]:
# mount google drive and secure GPU
import os, os.path, datetime
import keras
from google.colab import drive, runtime
import time
drive.mount('/content/drive')

"""some variables"""

data_name = "10k_1000_250_0"
subjects = [1,2,3,8,9,20,21,24,28,32,33,35,36,42,45,49,55,79,108,110,111,122,131,133]
test_subjects = [1,2,3,8,9,20] # if empty it will not use test subjects
learning_rate = 0.00001 #[] is default of 0.001

window_length = 10000
trials = [1,2,3]
model_to_use = 'resnet50_12channels' #'parallel_channels_LSTM, 'parallel_channels_conv1D', 'resnet50', 'resnet50_12channels'
epochs = 3000

flag_shuffle_train = 0 # maybe this can go, if we are shuffling subjects beforehand anyways
flag_shuffle_subjects = 0
flag_plot = 0
flag_top_5_accuracy = 0

output_variable = "subject_id" #"speed", "subject_id", "seconds_10k"
percentage_of_data_to_be_used = 1 # number between 0 and 100. 100 is all data, everything else takes the percentage


val_split = 0.2
test_split = 0.2

dropout = 0
batch_size = 32

early_stopping_min_delta = 0
early_stopping_patience = 2000
reinitialize_epochs = 1000


kernel_size_convolution = 250
input_model = keras.Input(shape = (100,60,12,1)) #keras.Input(shape = (126,40,12,1))  #keras.Input(shape = (10000,12)) # input for first layer
input_resnet = keras.Input(shape=(126,40,12)) # input after my own layers into the resnet
resnet_trainable = True
weights_to_use = 'trial_3_1st_calc2' #'imagenet', None




# import PredictPower_functions
%cd /content/drive/My\ Drive/Running Plantiga Project/Code
import script_classification as script_classification
#import PredictPower_functions as ppf
#import openloop_algorithms as oa


#check if GPU is available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))


%cd /content/drive/My\ Drive/Running Plantiga Project
dir_root = os.getcwd() + '/'

#This is if you want to load all data into google cloud first

try:
  # create all directories in google cloud storage
  os.mkdir('/content/Running Plantiga Project/')
  os.mkdir('/content/Running Plantiga Project/Code/')
  os.mkdir('/content/Running Plantiga Project/Data/')
  os.mkdir('/content/Running Plantiga Project/Data/Prepared/')
  os.mkdir('/content/Running Plantiga Project/Data/Prepared/tfrecords/')
  #os.mkdir('/content/Running Plantiga Project/Data/Prepared/tfrecords/10k_1000_250_0/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/models_info/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/models_trained/')
except:
  print('Paths already exist')


#!cp '/content/drive/MyDrive/Running Plantiga Project/' -R '/content/'

# move all relevant code and data
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Code/script_classification.py' '/content/Running Plantiga Project/Code/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Code/functions_classification.py' '/content/Running Plantiga Project/Code/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Subject_seconds_scores.csv' '/content/Running Plantiga Project/Data/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Prepared/tfrecords/10k_1000_250_0/' '/content/Running Plantiga Project/Data/Prepared/tfrecords/'

if weights_to_use != (None and 'imagenet'):
  !gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_trained/{weights_to_use}.h5' '/content/Running Plantiga Project/Data/Results/models_trained/'

# unmount google drive
#from google.colab import drive
#drive.flush_and_unmount()


# import PredictPower_functions
%cd /content/Running Plantiga Project/Code
import script_classification as script_classification
#import PredictPower_functions as ppf
#import openloop_algorithms as oa

%cd /content/Running Plantiga Project
dir_root = os.getcwd() + '/'





#check if GPU is available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))







name_ID = script_classification.script_classification(data_name, subjects, val_split, test_split, flag_shuffle_train, flag_plot, output_variable,
                          epochs, dropout, batch_size, early_stopping_min_delta, early_stopping_patience,
                          dir_root, trials, window_length, kernel_size_convolution, model_to_use,
                          weights_to_use, input_model, input_resnet, resnet_trainable, reinitialize_epochs, flag_shuffle_subjects,
                          flag_top_5_accuracy, test_subjects, learning_rate, percentage_of_data_to_be_used)

# only if using google cloud storage to afterwards load the data over to drive (which usually does not work anyways)

# remount drive and move my result files
#drive.mount('/content/drive')
!gsutil -m cp -r '/content/Running Plantiga Project/Data/Results/models_info/{name_ID}.pkl' '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_info/'
!gsutil -m cp -r '/content/Running Plantiga Project/Data/Results/models_trained/{name_ID}.h5' '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_trained/'



time.sleep(10)
!kill -9 -1





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Running Plantiga Project/Code
Found GPU at: /device:GPU:0
/content/drive/My Drive/Running Plantiga Project
Paths already exist
Copying file:///content/drive/MyDrive/Running Plantiga Project/Code/script_classification.py...
/ [1/1 files][ 11.7 KiB/ 11.7 KiB] 100% Done                                    
Operation completed over 1 objects/11.7 KiB.                                     
Copying file:///content/drive/MyDrive/Running Plantiga Project/Code/functions_classification.py...
/ [1/1 files][ 25.9 KiB/ 25.9 KiB] 100% Done                                    
Operation completed over 1 objects/25.9 KiB.                                     
Copying file:///content/drive/MyDrive/Running Plantiga Project/Data/Subject_seconds_scores.csv...
/ [1/1 files][  4.9 KiB/  4.9 KiB] 100% Done                                    
Operation completed 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Streaming output truncated to the last 5000 lines.
108/108 [==============================] - 7s 68ms/step - loss: 9.5284e-07 - accuracy: 1.0000 - val_loss: 0.4492 - val_accuracy: 0.8660
Epoch 1752/3000
108/108 [==============================] - ETA: 0s - loss: 3.8791e-06 - accuracy: 1.0000
Epoch 1752: val_accuracy did not improve from 0.97644
108/108 [==============================] - 7s 68ms/step - loss: 3.8791e-06 - accuracy: 1.0000 - val_loss: 0.4471 - val_accuracy: 0.8704
Epoch 1753/3000
108/108 [==============================] - ETA: 0s - loss: 5.7900e-07 - accuracy: 1.0000
Epoch 1753: val_accuracy did not improve from 0.97644
108/108 [==============================] - 7s 68ms/step - loss: 5.7900e-07 - accuracy: 1.0000 - val_loss: 0.4511 - val_accuracy: 0.8689
Epoch 1754/3000
108/108 [==============================] - ETA: 0s - loss: 1.1203e-06 - accuracy: 1.0000
Epoch 1754: val_accuracy did not improve from 0.97644
108/108 [==============================] - 7s 68ms/step - loss: 

8/8 [==============================] - 2s 130ms/step
Test_accuracy: [0.212910458445549, 0.9453781247138977]
1/1 [==============================] - 1s 745ms/step
Accuracy
0.9453781512605042
Confusion Matrix
[[47  0  0  0  0  0  6  0  0  0  0]
 [ 0 42  0  0  0  0  0  0  0  0  1]
 [ 0  0 26  0  0  0  0  0  1  4  0]
 [ 0  0  0 40  0  0  0  0  0  0  0]
 [ 0  0  0  0 41  0  0  0  0  0  0]
 [ 0  0  0  0  0 29  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]
saved model as as: 10k_1000_250_0_resnet50_12channels_subject_id_True_trial_3_1st_calc2_2023-10-04 19:49:28_transfer
subjects: [1, 2, 3, 8, 9, 20, 21, 24, 28, 32, 33, 35, 36, 42, 45, 49, 55, 79, 108, 110, 111, 122, 131, 133]
test subjects: [1, 2, 3, 8, 9, 20]
Copying file:///content/Running Plantiga Project/Data/Results/models_info/10k_1000_250_0_resnet50_12channels_subject_id_True_trial_3_1st_

NameError: ignored